In [42]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
X = 2 * np.random.rand(500, 2)
np.random.randn(500, 1)
y = 1 + 2 * X[:,0:1] + 3* X[:,1:] + np.random.randn(500, 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X.shape, y.shape
X[0]

array([0.53090474, 0.81305652])

In [24]:
X = np.c_[np.ones_like(X), X]
def hypothesis(X, theta):
    return np.dot(X, theta)

def gradient(X, y, theta):
    h = hypothesis(X, theta)
    grad = np.dot(X.transpose(), (h - y))
    return grad

def cost(X, y, theta):
    h = hypothesis(X, theta)
    J = np.dot((h - y).transpose(), (h - y))
    J /= 2
    return J[0]

def create_mini_batches(X, y, batch_size):
    mini_batches = []
    data = np.hstack((X, y))
    np.random.shuffle(data)
    n_minibatches = data.shape[0] // batch_size
    i = 0
    for i in range(n_minibatches + 1):
        mini_batch = data[i * batch_size:(i + 1)*batch_size, :]
        X_mini = mini_batch[:, :-1]
        Y_mini = mini_batch[:, -1].reshape((-1, 1))
        mini_batches.append((X_mini, Y_mini))
    if data.shape[0] % batch_size != 0:
        mini_batch = data[i * batch_size:data.shape[0]]
        X_mini = mini_batch[:, :-1]
        Y_mini = mini_batch[:, -1].reshape((-1, 1))
        mini_batches.append((X_mini, Y_mini))
    return mini_batches

def gradientDescent(X, y, learning_rate = 0.001, batch_size = 50):
    theta = np.zeros((X.shape[1], 1))
    error_list = []
    max_iters = 3
    for itr in range(max_iters):
        mini_batches = create_mini_batches(X, y, batch_size)
        for mini_batch in mini_batches:
            X_mini, y_mini = mini_batch
            theta = theta - learning_rate * gradient(X_mini, y_mini, theta)
            error_list.append(cost(X_mini, y_mini, theta))
  
    return theta, error_list
theta, error_list = gradientDescent(X_train, y_train)

In [30]:
#still struggling to implement mse for evaluation
error = np.sum(np.abs(y_test - y_pred) / y_test.shape[0])
print("Error: ", error)

Error:  0.9253203741644246


In [2]:
import pandas as pd
import sklearn
from sklearn.datasets import fetch_openml
mnist = fetch_openml(name='mnist_784', version=1)
X, y = mnist['data'], mnist['target']
y = y.astype(np.int8)
X_train, y_train, X_test, y_test = X[:60000], y[:60000], X[60000:], y[60000:]
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((60000, 784), (60000,), (10000, 784), (10000,))

In [3]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(solver='liblinear')
y_train_even = (y_train % 2 == 0)
y_test_even = (y_test % 2 == 0)

In [4]:
log_reg.fit(X=X_train, y=y_train_even)

LogisticRegression(solver='liblinear')

In [5]:
y_pred = log_reg.predict(X_train)
num_correct = sum(y_pred==y_train_even)
print('train accuracy: ', num_correct / len(y_pred))

train accuracy:  0.9039


In [16]:
y_pred = log_reg.predict(X_test)
num_correct = sum(y_pred==y_test_even)
print('test accuracy: ', num_correct / len(y_pred))

test accuracy:  0.9013


In [7]:
def convert_digit(digit):
    if digit<3:
        return 0
    elif digit>6:
        return 2
    else:
        return 1

In [8]:
y_train_Q5 = [convert_digit(digit) for digit in y_train]
y_test_Q5 = [convert_digit(digit) for digit in y_test]

In [14]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
X_train_minmax = min_max_scaler.fit_transform(X_train)
softmax_reg = LogisticRegression(multi_class='multinomial', C=100)
softmax_reg.fit(X=X_train_minmax, y=y_train_Q5)

C:\Users\Tbolt\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=100, multi_class='multinomial')

In [17]:
y_pred = softmax_reg.predict(X_train)
num_correct = sum(y_pred==y_train_Q5)
print('train accuracy: ', num_correct / len(y_pred))

train accuracy:  0.8052666666666667


In [19]:
y_pred = softmax_reg.predict(X_test)
num_correct = sum(y_pred==y_test_Q5)
print('test accuracy: ', num_correct / len(y_pred))

test accuracy:  0.7978
